In [1]:
import torch
import torchsummary # 모델 구조 보기좋게 출력
import torchvision
from torchvision import models
from torchvision import utils
from torchvision.utils import *

import numpy as np
from pathlib import Path

# 환경

In [2]:
# device 세팅

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


# 모델

In [3]:
def ResModule(in_chnnel, out_chnnel):
    return torch.nn.Sequential(
        torch.nn.Conv2d(in_chnnel, out_chnnel,(3, 3), padding = 1),
        torch.nn.BatchNorm2d(out_chnnel),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d((2, 2))
    )

model = torch.nn.Sequential(
    ResModule(3, 64),
    ResModule(64, 128),
    ResModule(128, 256),
    ResModule(256, 128),
    torch.nn.AdaptiveMaxPool2d(output_size=(1, 1)),
    torch.nn.Flatten(),
    torch.nn.Linear(128, 10, bias = True)
    
)


model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16,(3, 3), padding = 1),
    torch.nn.BatchNorm2d(16),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2, 2)),
    
    torch.nn.Conv2d(16, 16,(3, 3), padding = 1),
    torch.nn.BatchNorm2d(16),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2, 2)),
    
    torch.nn.Conv2d(16, 32,(3, 3), padding = 1),
    torch.nn.BatchNorm2d(32),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2, 2)),
    
    torch.nn.Conv2d(32, 16,(3, 3), padding = 1),
    torch.nn.BatchNorm2d(16),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2, 2)),
    
    torch.nn.AdaptiveMaxPool2d(output_size=(1, 1)),
    torch.nn.Flatten(),
    torch.nn.Linear(16, 10, bias = True)
)

In [4]:
model.to(device)
torchsummary.summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             448
       BatchNorm2d-2           [-1, 16, 64, 64]              32
              ReLU-3           [-1, 16, 64, 64]               0
         MaxPool2d-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 16, 32, 32]           2,320
       BatchNorm2d-6           [-1, 16, 32, 32]              32
              ReLU-7           [-1, 16, 32, 32]               0
         MaxPool2d-8           [-1, 16, 16, 16]               0
            Conv2d-9           [-1, 32, 16, 16]           4,640
      BatchNorm2d-10           [-1, 32, 16, 16]              64
             ReLU-11           [-1, 32, 16, 16]               0
        MaxPool2d-12             [-1, 32, 8, 8]               0
           Conv2d-13             [-1, 16, 8, 8]           4,624
      BatchNorm2d-14             [-1, 1

# 데이터 셋

In [5]:
import os
import pathlib
from torchvision import transforms
from torch.utils.data import DataLoader

pathlib.Path('./train').mkdir(parents = True, exist_ok=True)
pathlib.Path('./test').mkdir(parents = True, exist_ok=True)

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize([64, 64]),
    torchvision.transforms.Normalize((0.485*255, 0.456*255, 0.406*255), (0.229, 0.224, 0.225))
])
train_dataset = torchvision.datasets.STL10('/train', split='train', download=True, transform=transform)
test_dataset = torchvision.datasets.STL10('/test', split='test', download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 학습

In [38]:
def val(model, params):
    device = params["device"]
    validation_dataloader = params["val_dataloader"]
    loss_function = params["loss_function_generator"]()
    model.to(device)
    
    total = 0
    right = 0
    loss = 0

    for input_batch, label_batch in validation_dataloader:
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)
        
        length = len(input_batch)
        total += length
        
        predict = model(input_batch)
        new_loss = loss_function(predict, label_batch)
        
        loss = new_loss+ (new_loss-loss)*length/total
        predict_class = torch.max(predict, 1)[1]
        right += (predict_class == label_batch).sum().item()
        
    acc = right/total
    print(right, total)
    return acc, loss
 


def train(model, params):
    device = params["device"]
    train_dataloader = params["train_dataloader"] 
    val_dataloader = params["val_dataloader"]
    
    epoch = params["epoch"]
    lr = params["learning_rate"]
    
    optimizer = params["optimizer_generator"](model.parameters(), lr = lr)
    loss_function = params["loss_function_generator"]().to(device)
    
    model.to(device)
    
    for e in range(epoch):
        for input_batch, label_batch in train_dataloader:
            input_batch = input_batch.to(device)
            label_batch = label_batch.to(device)
            optimizer.zero_grad()
            predict = model(input_batch)
            loss = loss_function(predict, label_batch)
            loss.backward()
            optimizer.step()
        
        print(val(model, params))

In [40]:
params = {
    "device":device,
    
    "train_dataloader":train_dataloader,
    "val_dataloader":test_dataloader,

    "epoch": 10,
    "learning_rate":0.0001,
    
    "optimizer_generator": torch.optim.Adam,
    "loss_function_generator":torch.nn.CrossEntropyLoss
}

train(model, params)

2129 8000
(0.266125, tensor(1.9365, device='cuda:0', grad_fn=<AddBackward0>))
2251 8000
(0.281375, tensor(1.9042, device='cuda:0', grad_fn=<AddBackward0>))
2247 8000
(0.280875, tensor(1.8579, device='cuda:0', grad_fn=<AddBackward0>))
2440 8000
(0.305, tensor(1.8470, device='cuda:0', grad_fn=<AddBackward0>))
2518 8000
(0.31475, tensor(1.8057, device='cuda:0', grad_fn=<AddBackward0>))
2647 8000
(0.330875, tensor(1.7854, device='cuda:0', grad_fn=<AddBackward0>))
2721 8000
(0.340125, tensor(1.7667, device='cuda:0', grad_fn=<AddBackward0>))
2742 8000
(0.34275, tensor(1.7401, device='cuda:0', grad_fn=<AddBackward0>))
2795 8000
(0.349375, tensor(1.6816, device='cuda:0', grad_fn=<AddBackward0>))
2794 8000
(0.34925, tensor(1.6701, device='cuda:0', grad_fn=<AddBackward0>))


In [41]:
params = {
    "device":device,
    
    "train_dataloader":train_dataloader,
    "val_dataloader":test_dataloader,

    "epoch": 10,
    "learning_rate":0.0001,
    
    "optimizer_generator": torch.optim.Adam,
    "loss_function_generator":torch.nn.CrossEntropyLoss
}
model = models.resnet50()
train(model, params)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.00 GiB total capacity; 9.25 GiB already allocated; 0 bytes free; 9.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
import torch
import torch.nn as nn
from torch import optim

In [8]:
lr = 0.0001
num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss().to(device)

In [9]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    'test_dataloader': test_dataloader,
    'device':device
}

In [10]:
def train(model, params):
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    test_dataloader=params["test_dataloader"]
    device=params["device"]

    for epoch in range(0, num_epochs):
        for i, data in enumerate(train_dataloader, 0):
            # train dataloader 로 불러온 데이터에서 이미지와 라벨을 분리
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 이전 batch에서 계산된 가중치를 초기화
            optimizer.zero_grad() 

            # forward + back propagation 연산
            outputs = model(inputs)
            train_loss = loss_function(outputs, labels)
            train_loss.backward()
            optimizer.step()

        # test accuracy 계산
        total = 0
        correct = 0
        accuracy = []
        
        for i, data in enumerate(test_dataloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 결과값 연산
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss = loss_function(outputs, labels).item()
            accuracy.append(100 * correct/total)

        # 학습 결과 출력
        print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch+1, num_epochs, train_loss.item(), test_loss, 100*correct/total))

In [11]:
train(model, params)

Epoch: 1/5, Train loss: 2.218821, Test loss: 2.384054, Accuracy: 10.00
Epoch: 2/5, Train loss: 2.280489, Test loss: 2.325967, Accuracy: 10.00
Epoch: 3/5, Train loss: 2.298495, Test loss: 2.316422, Accuracy: 12.88
Epoch: 4/5, Train loss: 2.233734, Test loss: 2.285816, Accuracy: 13.19
Epoch: 5/5, Train loss: 2.257638, Test loss: 2.123544, Accuracy: 18.90


In [12]:
model[]

SyntaxError: invalid syntax (2951394079.py, line 1)